In this notebook, I am comparing my hepMC3 file output to my custom ROOT Tree on TParticle output. 

To do this, I am looking at all pions. I select all their mothers, and count their PDG's. This is a bit more robust than just counting all particles, since it also tests if our custom output's mother relations are working right.

We can compare these two because I used the same random seed when generating the hepMC3 and ROOT Tree output.

In [1]:
import ROOT as rt
import pyhepmc

In [6]:
mother_counts = {}
excluded_pdgs = {
}

with pyhepmc.open("output1.hepmc", "r") as f:
    for event in f:
        for p in event.particles:
            if abs(p.pid) == 211 and p.production_vertex:
                for mom in p.production_vertex.particles_in:
                    pdg = mom.pid
                    if pdg in excluded_pdgs:
                        continue
                    if pdg in mother_counts:
                        mother_counts[pdg] += 1
                    else:
                        mother_counts[pdg] = 1

# Display top 10
print("Mother PDG Code | Count")
print("------------------------")
for pdg, count in sorted(mother_counts.items(), key=lambda x: x[1], reverse=True)[0:15]:
    print(f"{pdg:<17} | {count}")

Mother PDG Code | Count
------------------------
21                | 61205
2                 | 8154
113               | 6846
223               | 6076
1                 | 5025
2101              | 4950
2203              | 3426
213               | 3409
-213              | 3191
310               | 3038
221               | 1496
2103              | 1477
-1                | 1408
-2                | 1381
-3                | 699


In [5]:
# Open the ROOT file and get the tree
root_file = rt.TFile("output_tparticle.root")
tree = root_file.Get("EventTree")

mother_counts_root = {}
excluded_pdgs = {
}

pions = 0
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", rt.AddressOf(particles))

# Loop over entries
for entry in range(tree.GetEntries()):
    tree.GetEntry(entry)
    n_particles = particles.GetEntriesFast()

    for i in range(n_particles):
        part = particles.At(i)
        if abs(part.GetPdgCode()) == 211:  # pion
            pions += 1
            idx1 = part.GetMother(0)
            idx2 = part.GetMother(1)

            for idx in [idx1, idx2]:
                if 0 <= idx < n_particles:
                    mom = particles.At(idx)
                    pdg_mom = mom.GetPdgCode()
                    if pdg_mom in excluded_pdgs:
                        continue
                    mother_counts_root[pdg_mom] = mother_counts_root.get(pdg_mom, 0) + 1

# Print summary
print(f"Total pions: {pions}")
print("ROOT TTree - Top 15 Mother PDG Counts for pions:")
for pdg, count in sorted(mother_counts_root.items(), key=lambda x: x[1], reverse=True)[:15]:
    print(f"{pdg:<10} | {count}")

Total pions: 43105
ROOT TTree - Top 15 Mother PDG Counts for pions:
90         | 29297
2          | 7991
113        | 6846
223        | 6076
1          | 5004
2101       | 4950
2203       | 3426
213        | 3409
-213       | 3191
310        | 3038
221        | 1496
2103       | 1477
-1         | 1408
-2         | 1381
-3         | 699


the issue is pythia is not accurate down to the quark and gluon level. hence, lets not include quarks and gluons in our comparison. also, let's not consider the PDG of 90, which likely is an internal pythia reference

In [7]:
excluded_pdgs = {
    1, 2, 3, -1, -2, -3, 21, 90,
    2101, 2103, 2203, 2201, 1103, 1203, 1303, -2101, -2103, -2203  
}

In [8]:
mother_counts = {}

with pyhepmc.open("output1.hepmc", "r") as f:
    for event in f:
        for p in event.particles:
            if abs(p.pid) == 211 and p.production_vertex:
                for mom in p.production_vertex.particles_in:
                    pdg = mom.pid
                    if pdg in excluded_pdgs:
                        continue
                    if pdg in mother_counts:
                        mother_counts[pdg] += 1
                    else:
                        mother_counts[pdg] = 1

# Display top 10
print("Mother PDG Code | Count")
print("------------------------")
for pdg, count in sorted(mother_counts.items(), key=lambda x: x[1], reverse=True)[0:15]:
    print(f"{pdg:<17} | {count}")

Mother PDG Code | Count
------------------------
113               | 6846
223               | 6076
213               | 3409
-213              | 3191
310               | 3038
221               | 1496
-313              | 520
323               | 518
-323              | 517
313               | 510
2224              | 452
3122              | 417
-3122             | 297
331               | 284
1114              | 226


In [9]:
# Open the ROOT file and get the tree
root_file = rt.TFile("output_tparticle.root")
tree = root_file.Get("EventTree")

mother_counts_root = {}

pions = 0
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", rt.AddressOf(particles))

# Loop over entries
for entry in range(tree.GetEntries()):
    tree.GetEntry(entry)
    n_particles = particles.GetEntriesFast()

    for i in range(n_particles):
        part = particles.At(i)
        if abs(part.GetPdgCode()) == 211:  # pion
            pions += 1
            idx1 = part.GetMother(0)
            idx2 = part.GetMother(1)

            for idx in [idx1, idx2]:
                if 0 <= idx < n_particles:
                    mom = particles.At(idx)
                    pdg_mom = mom.GetPdgCode()
                    if pdg_mom in excluded_pdgs:
                        continue
                    mother_counts_root[pdg_mom] = mother_counts_root.get(pdg_mom, 0) + 1

# Print summary
print(f"Total pions: {pions}")
print("ROOT TTree - Top 15 Mother PDG Counts for pions:")
for pdg, count in sorted(mother_counts_root.items(), key=lambda x: x[1], reverse=True)[:15]:
    print(f"{pdg:<10} | {count}")

Total pions: 43105
ROOT TTree - Top 15 Mother PDG Counts for pions:
113        | 6846
223        | 6076
213        | 3409
-213       | 3191
310        | 3038
221        | 1496
-313       | 520
323        | 518
-323       | 517
313        | 510
2224       | 452
3122       | 417
-3122      | 297
331        | 284
1114       | 226


### **great! we see that the PDG's are the same -- our TParticle ROOT Tree seems to be working well.**

random cool visualization

In [1]:
import pyhepmc

# pyhepmc.open can read most HepMC formats using auto-detection
with pyhepmc.open("output1.hepmc") as f:
    event = f.read()

event

<GenEvent momentum_unit=1, length_unit=0, event_number=0, particles=164, vertices=82, run_info=GenRunInfo(tools=[], weight_names=[], attributes={})>